# Instalo e importo librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=4e86f44017fd819be8374a6dd58becab84683a6adccb49e4a27acff094dfb304
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creo sesión de spark y leo csv

In [ ]:
downloaded = drive.CreateFile({'id':"1gDD0Mn9bYoj4RQlPb7V2_3mJ_-Ikmit8"})
downloaded.GetContentFile('GooglePlayStore.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore.csv', header=True, inferSchema=True)
apps_details = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Ejercicio 32

Martín tiene un celular de gama baja, y tiene poca capacidad de almacenamiento. ¿Cuál es la app de menor tamaño que puede descargar desde google play? (⭐)

Hay apps que como size tienen 'Varies with device'. Estos datos no me servirán para lo que busco así que filtraré esto.

Hay otra con datos mal cargados que figura con size '1,000+'. Puede hallarse fácilmente ya que es la única con rating absurdo (rating 19 cuando es un valor entre 0 y 5). Las demás tienen su tamaño en Mb o kb, y esta no, por lo que no me servirá (no sé si el dato está en mb o kb) y la filtraré.

Me voy a quedar con las apps y sus respectivos tamaños (mapeando, dejo esto como clave-valor).

Por como filtro los datos, size de cada app es algo tipo 'x.xM' o 'xk', es decir, tamaños expresados en Megabytes (algunos con coma) o kilobytes. Lo pasaré a bytes para tener todo unificado (también en el map).

Hay apps que están repetidas en el csv. La mayoría de las apps repetidas tiene el dato de su tamaño igual, pero desconozco si esto sucede en TODOS los casos, quizás haya discrepancias. Por esto, las consideraré erorres y me quedaré con el tamaño que aparece último como el válido (con el reduceByKey).

Por último, calculo la app con menor tamaño con un reduce. Muestro clave-valor pero bien podría haberme quedado solo con el nombre de la app, haciendo [0] al final de todo.

In [ ]:
def map_to_bytes(size):
  if size[-1] == 'M':
    size = float(size.replace('M', ''))*1048576
  elif size[-1] == 'k':
    size = float(size.replace('k', ''))*1024
  return size


In [ ]:
apps_details.filter(lambda x: (x.Size != '1,000+') & (x.Size != 'Varies with device')).map(lambda x: (x.App, map_to_bytes(x.Size)))\
.reduceByKey(lambda a,b: b).reduce(lambda x,y: x if x[1] < y[1] else y) #con tamaño

('Essential Resources', 8704.0)

La app de menor tamaño que Martín puede descargar desde google play es 'Essential Resources' y pesa 8704 bytes (8.5kb)